In [57]:
import qiskit
from qiskit import IBMQ
import os
API_TOKEN = os.environ.get('IBM_Q_TOKEN')
IBMQ.save_account(API_TOKEN)

from qiskit import QuantumRegister
from qiskit import QuantumRegister
from qiskit import QuantumCircuit
from qiskit import ClassicalRegister
from qiskit import execute




## Deutsch-Jozsa (Deutsch's Algorithm)

In [58]:
from qiskit import Aer

### Execute locally

In [59]:
# execute circuit and either display a histogram of the results
def execute_locally(qc, draw_circuit=False, show_results=False):
    # Compile and run the Quantum circuit on a simulator backend
    backend_sim = Aer.get_backend('qasm_simulator')
    job_sim = execute(qc, backend_sim)
    result_sim = job_sim.result()
    result_counts = result_sim.get_counts(qc)
    
    if draw_circuit or show_results: # Print the results
        print("simulation: ", result_sim, result_counts)
    
    if draw_circuit: # draw the circuit
        draw(qc)
    elif show_results: # or show the results
        show_results(result_counts)
    return result_counts

### Execute remotely

In [63]:
from qiskit.backends.ibmq import least_busy
import time
# Compile and run on a real device backend
def execute_remotely(qc, draw_circuit=False, show_results=False):
    if draw_circuit: # draw the circuit
        draw(qc)
    try:
        # select least busy available device and execute.
        least_busy_device = least_busy(IBMQ.backends(simulator=False))
        print("Running on current least busy device: ", least_busy_device)

        # running the job
        job_exp = execute(qc, backend=least_busy_device, shots=1024, max_credits=10)

        lapse, interval = 0, 10
        while job_exp.status().name != 'DONE':
            print('Status @ {} seconds'.format(interval * lapse))
            print(job_exp.status())
            time.sleep(interval)
            lapse += 1
        print(job_exp.status())
        exp_result = job_exp.result()
        result_counts = exp_result.get_counts(qc)

        # Show the results
        print("experiment: ", exp_result, result_counts)
        if show_results: # show the results
            show_results(result_counts)
        return result_counts
    except:
        print("All devices are currently unavailable.")
        return {}

ModuleNotFoundError: No module named 'qiskit.backends'

## Building the circuit

In [64]:
def new_circuit(size):
    # Create a Quantum Register with size qubits
    qr = QuantumRegister(size)

    # Create a Classical Register with size bits
    cr = ClassicalRegister(size)

    # Create a Quantum Circuit acting on the qr and cr register
    return qr, cr, QuantumCircuit(qr, cr)

In [65]:
def deutsch(black_box):
    qr, cr, circuit = new_circuit(2)
    circuit.x(qr[1]) # X gate on qubit 1 (bit flip)
    circuit.h(qr) # Hadamard on both qubits
    
    black_box(qr, cr, circuit)
    
    circuit.h(qr[0]) # Hadamard on interesting qubit
    circuit.measure(qr[0], cr[0]) # measure the specific qubit
    
    results = execute_locally(circuit, draw_circuit=False, show_results=False) # silent mode
    return get_deutsch_verdict(results)

In [66]:
def quantum_oracle_1(qr, cr, circuit):
    pass

In [67]:
def quantum_oracle_2(qr, cr, circuit):
    circuit.cx(qr[0], qr[1])

In [68]:
def quantum_oracle_3(qr, cr, circuit):
    circuit.cx(qr[0], qr[1])
    circuit.cx(qr[1], qr[0])
    circuit.cx(qr[0], qr[1])

In [69]:
def quantum_oracle_4(qr, cr, circuit):
    circuit.z(qr[1])
    circuit.cx(qr[0], qr[1])

In [70]:
deutsch(quantum_oracle_1)

NameError: name 'get_deutsch_verdict' is not defined